Use Prochlorococcus example to figure out how to match data to a place (I will ultimately do BATS)\
Krista Longnecker, 15 June 2025


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pycmap
from pycmap.viz import plot_timeseries
import os #sometimes useful
from importlib import reload

In [27]:
#get the API from the CMAP website scripts have the API in config/config.py
#api = pycmap.API(token='8147c3f6-e99c-47f5-9d72-1290ae601ebb', vizEngine='plotly')

In [28]:
#scripts are from the Simons CMAP article (case study), linked from LOM paper

In [35]:
#need to run this command after any edits are made to common.py; otherwise the notebook just reads in whatever was read in first
#I am not entirely sure what is going on, but I gather I can import the py file and then reload it as needed
import common
reload(common)

<module 'common' from 'C:\\Users\\klongnecker\\Documents\\Dropbox\\GitHub\\tryCMAP\\matchToBATS\\common.py'>

In [2]:
#first, need to figure out how to run .py scripts from jupyter notebook
#scripts are all in matchToBATS
%run collect.py


********************************
********************************


********************************
********************************



In [ ]:
%run colocalize.py

0 / 39
	2025-06-16 07:29:14.458630: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
1 / 39
	2025-06-16 07:29:14.462672: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
2 / 39
	2025-06-16 07:29:14.465583: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
3 / 39
	2025-06-16 07:29:14.470102: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
4 / 39
	2025-06-16 07:29:14.473498: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
5 / 39
	2025-06-16 07:29:14.476694: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
6 / 39
	2025-06-16 07:29:14.479624: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
7 / 39
	2025-06-16 07:29:14.482672: Colocalizing tblSST_AVHRR_OI_NRT with ./data\tblD284_AMT15_Flow_Cytometry.csv ...
8 / 39
	2025-06-16 07:29:14.485275: Colocalizing tblSST_

In [35]:
#have an issue, see if I can work through the steps here to figure out whyb

In [ ]:
#Stick some code below this spot as a holding zone, but put in a stop so the notebook doesn't try and run the bits as they may or may not work.

In [ ]:
 raise SystemExit("Stop execution here")

In [18]:
import os, glob
import concurrent.futures
import pycmap
from config.config import API_KEY
from settings import DATA_DIR, COLOCALIZED_DIR
from common import halt, makedir, environmental_datasets
import pandas as pd
import datetime
from dateutil.parser import parse



def cyano_csv_files(cyanoDir):
    """
    Returns a list of path to csv files that hold observations of cyanobacteria.
    """
    return glob.glob(f"{cyanoDir}*.csv")


def add_env_columns(df, envs):
    """
    Adds new columns to the dataframe form each environmental variable.
    """
    for env in envs.values():
        for v in env.get("variables"):
            if v not in df.columns: df[v] = None
    return df
    

In [19]:
def add_env_temporal_coverage(api, envs):
    """
    Adds new entries to the envs dictionary indicating the temporal coverage of each environmental dataset.
    """
    for table, env in envs.items():
        df = api.query(f"SELECT MIN([time]) startTime, MAX([time]) endTime FROM {table}")
        if len(df) > 0:
            envs[table]["startTime"] = df.loc[0, "startTime"]
            envs[table]["endTime"] = df.loc[0, "endTime"]
    return envs

In [20]:
DATA_DIR = "./data/"           
cyanoFiles = cyano_csv_files(DATA_DIR)

In [21]:
cyanoFiles

['./data\\tblD284_AMT15_Flow_Cytometry.csv',
 './data\\tblD294_AMT16_Flow_Cytometry.csv',
 './data\\tblD299_AMT17_Flow_Cytometry.csv',
 './data\\tblD371_AMT21_flow_cytometry.csv',
 './data\\tblDY110_AMT29_flow_cytometry.csv',
 './data\\tblFlombaum.csv',
 './data\\tblGlobal_PicoPhytoPlankton.csv',
 './data\\tblJC039_AMT19_flow_cytometry.csv',
 './data\\tblJC053_AMT20_flow_cytometry.csv',
 './data\\tblJC079_AMT22_flow_cytometry.csv',
 './data\\tblJR15001_AMT25_flow_cytometry.csv',
 './data\\tblJR19980514_AMT06_Flow_Cytometry.csv',
 './data\\tblJR20030512_AMT12_Flow_Cytometry.csv',
 './data\\tblJR20030910_AMT13_Flow_Cytometry.csv',
 './data\\tblJR20040428_AMT14_Flow_Cytometry.csv',
 './data\\tblJR20081003_AMT18_flow_cytometry.csv',
 './data\\tblJR20131005_AMT23_flow_cytometry.csv',
 './data\\tblJR20140922_AMT24_flow_cytometry.csv',
 './data\\tblSeaFlow_v1_5.csv']

In [22]:
    api = pycmap.API(token=API_KEY)
    makedir(COLOCALIZED_DIR)
    envs = environmental_datasets()        

In [23]:
envs


{'tblSST_AVHRR_OI_NRT': {'variables': ['sst'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False},
 'tblCHL_REP': {'variables': ['chl'],
  'tolerances': [4, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False},
 'tblSSS_NRT': {'variables': ['sss'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False},
 'tblModis_PAR': {'variables': ['PAR'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False},
 'tblAltimetry_REP': {'variables': ['sla', 'adt', 'ugosa', 'vgosa'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False},
 'tblPisces_NRT': {'variables': ['NO3', 'PO4', 'Fe', 'O2', 'Si', 'PP'],
  'tolerances': [4, 0.5, 0.5, 5],
  'hasDepth': True,
  'isClimatology': False},
 'tblWOA_Climatology': {'variables': ['density_WOA_clim',
   'salinity_WOA_clim',
   'nitrate_WOA_clim',
   'phosphate_WOA_clim',
   'silicate_WOA_clim',
   'oxygen_WOA_clim'],
  'tolerances': [1

In [24]:
#this adds time information to an existing envs dictionary
envs = add_env_temporal_coverage(api, envs)

In [25]:
envs

{'tblSST_AVHRR_OI_NRT': {'variables': ['sst'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False,
  'startTime': '1981-09-01T00:00:00.000Z',
  'endTime': '2025-06-10T00:00:00.000Z'},
 'tblCHL_REP': {'variables': ['chl'],
  'tolerances': [4, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False,
  'startTime': '1998-01-01T00:00:00.000Z',
  'endTime': '2018-06-26T00:00:00.000Z'},
 'tblSSS_NRT': {'variables': ['sss'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False,
  'startTime': '2015-03-31T00:00:00.000Z',
  'endTime': '2022-05-01T00:00:00.000Z'},
 'tblModis_PAR': {'variables': ['PAR'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False,
  'startTime': '2010-01-01T00:00:00.000Z',
  'endTime': '2022-03-07T00:00:00.000Z'},
 'tblAltimetry_REP': {'variables': ['sla', 'adt', 'ugosa', 'vgosa'],
  'tolerances': [1, 0.25, 0.25, 5],
  'hasDepth': False,
  'isClimatology': False},
 'tblPisces_

In [16]:
cyanoFiles

['./data\\tblD284_AMT15_Flow_Cytometry.csv',
 './data\\tblD294_AMT16_Flow_Cytometry.csv',
 './data\\tblD299_AMT17_Flow_Cytometry.csv',
 './data\\tblD371_AMT21_flow_cytometry.csv',
 './data\\tblDY110_AMT29_flow_cytometry.csv',
 './data\\tblFlombaum.csv',
 './data\\tblGlobal_PicoPhytoPlankton.csv',
 './data\\tblJC039_AMT19_flow_cytometry.csv',
 './data\\tblJC053_AMT20_flow_cytometry.csv',
 './data\\tblJC079_AMT22_flow_cytometry.csv',
 './data\\tblJR15001_AMT25_flow_cytometry.csv',
 './data\\tblJR19980514_AMT06_Flow_Cytometry.csv',
 './data\\tblJR20030512_AMT12_Flow_Cytometry.csv',
 './data\\tblJR20030910_AMT13_Flow_Cytometry.csv',
 './data\\tblJR20040428_AMT14_Flow_Cytometry.csv',
 './data\\tblJR20081003_AMT18_flow_cytometry.csv',
 './data\\tblJR20131005_AMT23_flow_cytometry.csv',
 './data\\tblJR20140922_AMT24_flow_cytometry.csv',
 './data\\tblSeaFlow_v1_5.csv']

In [18]:
cyanoFile = cyanoFiles[1]
df = pd.read_csv(cyanoFile)
df = add_env_columns(df, envs)
dfs = [df.loc[i].to_frame().T for i in range(len(df))]

In [19]:
colocalizedDF  = pd.DataFrame({})

In [21]:
def match(df, api, envs, cyanoFile, rowCount):
    """
    Takes a single-row dataframe containing cyano observations and colocalizes with the 
    environmental variables included in the `envs` argument. The tolerance parametrs 
    are also included in the `envs` argument.
    """ 
    def get_month(dt):
        return parse(dt).month

    def shift_dt(dt, delta):
        delta = float(delta)
        dt = parse(dt)
        dt += datetime.timedelta(days=delta)
        return dt.strftime("%Y-%m-%d %H:%M:%S")

    def in_time_window(sourceDT, targetMinDT, targetMaxDT):
        targetMinDT = targetMinDT.split(".000Z")[0]
        targetMaxDT = targetMaxDT.split(".000Z")[0]
        return not (
                    parse(sourceDT) < parse(targetMinDT) or 
                    parse(sourceDT) > parse(targetMaxDT)
                    )

    def construc_query(table, env, t, lat, lon, depth):
        variables = env["variables"] 
        timeTolerance = env["tolerances"][0] 
        latTolerance = env["tolerances"][1] 
        lonTolerance = env["tolerances"][2]  
        depthTolerance = env["tolerances"][3]  
        hasDepth = env["hasDepth"] 
        isClimatology = env["isClimatology"]
        inTimeRange = True
        if not isClimatology:
            startTime = env["startTime"] #issue here, KeyError 'startTime'
            endTime = env["endTime"]    
            inTimeRange = in_time_window(t, startTime, endTime)
        selectClause = "SELECT " + ", ".join([f"AVG({v}) {v}" for v in variables]) + " FROM " + table
        timeClause = f" WHERE [time] BETWEEN '{shift_dt(t, -timeTolerance)}' AND '{shift_dt(t, timeTolerance)}' "
        if not inTimeRange or isClimatology: timeClause = f" WHERE [month]={get_month(t)} "
        latClause = f" AND lat BETWEEN {lat-latTolerance} AND {lat+latTolerance} "
        lonClause = f" AND lon BETWEEN {lon-lonTolerance} AND {lon+lonTolerance} "
        depthClause = f" AND depth BETWEEN {depth-depthTolerance} AND {depth+depthTolerance} "
        if not hasDepth: depthClause = ""                
        return selectClause + timeClause + latClause + lonClause + depthClause        


    if len(df) != 1: halt(f"Invalid dataframe input.\nExpected a single row dataframe but received {len(df)} rows.")
    rowIndex = df.index.values[0]
    df.reset_index(drop=True, inplace=True)
    t= df.iloc[0]["time"]
    lat = df.iloc[0]["lat"]
    lon = df.iloc[0]["lon"] 
    depth = 0
    if 'depth' in df.columns: depth = df.iloc[0]["depth"]
    for table, env in envs.items():
        print(f"{rowIndex} / {rowCount-1}\n\t{datetime.datetime.now()}: Colocalizing {table} with {cyanoFile} ...")
        query = construc_query(table, env, t, lat, lon, depth)
        matchedEnv = api.query(query)
        if len(matchedEnv)>0:
            for v in env["variables"]: df.at[0, v] = matchedEnv.iloc[0][v] 
    return df

In [33]:

def get_month(dt):
    return parse(dt).month

def shift_dt(dt, delta):
    delta = float(delta)
    dt = parse(dt)
    dt += datetime.timedelta(days=delta)
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def in_time_window(sourceDT, targetMinDT, targetMaxDT):
    targetMinDT = targetMinDT.split(".000Z")[0]
    targetMaxDT = targetMaxDT.split(".000Z")[0]
    return not (
                parse(sourceDT) < parse(targetMinDT) or 
                parse(sourceDT) > parse(targetMaxDT)
                )

In [34]:
def construc_query(table, env, t, lat, lon, depth):
    variables = env["variables"] 
    timeTolerance = env["tolerances"][0] 
    latTolerance = env["tolerances"][1] 
    lonTolerance = env["tolerances"][2]  
    depthTolerance = env["tolerances"][3]  
    hasDepth = env["hasDepth"] 
    isClimatology = env["isClimatology"]
    inTimeRange = True
    if not isClimatology:
        startTime = env["startTime"] #issue here, KeyError 'startTime'
        endTime = env["endTime"]    
        inTimeRange = in_time_window(t, startTime, endTime)
    selectClause = "SELECT " + ", ".join([f"AVG({v}) {v}" for v in variables]) + " FROM " + table
    timeClause = f" WHERE [time] BETWEEN '{shift_dt(t, -timeTolerance)}' AND '{shift_dt(t, timeTolerance)}' "
    if not inTimeRange or isClimatology: timeClause = f" WHERE [month]={get_month(t)} "
    latClause = f" AND lat BETWEEN {lat-latTolerance} AND {lat+latTolerance} "
    lonClause = f" AND lon BETWEEN {lon-lonTolerance} AND {lon+lonTolerance} "
    depthClause = f" AND depth BETWEEN {depth-depthTolerance} AND {depth+depthTolerance} "
    if not hasDepth: depthClause = ""                
    return selectClause + timeClause + latClause + lonClause + depthClause   

In [26]:
envs.items()

dict_items([('tblSST_AVHRR_OI_NRT', {'variables': ['sst'], 'tolerances': [1, 0.25, 0.25, 5], 'hasDepth': False, 'isClimatology': False, 'startTime': '1981-09-01T00:00:00.000Z', 'endTime': '2025-06-10T00:00:00.000Z'}), ('tblCHL_REP', {'variables': ['chl'], 'tolerances': [4, 0.25, 0.25, 5], 'hasDepth': False, 'isClimatology': False, 'startTime': '1998-01-01T00:00:00.000Z', 'endTime': '2018-06-26T00:00:00.000Z'}), ('tblSSS_NRT', {'variables': ['sss'], 'tolerances': [1, 0.25, 0.25, 5], 'hasDepth': False, 'isClimatology': False, 'startTime': '2015-03-31T00:00:00.000Z', 'endTime': '2022-05-01T00:00:00.000Z'}), ('tblModis_PAR', {'variables': ['PAR'], 'tolerances': [1, 0.25, 0.25, 5], 'hasDepth': False, 'isClimatology': False, 'startTime': '2010-01-01T00:00:00.000Z', 'endTime': '2022-03-07T00:00:00.000Z'}), ('tblAltimetry_REP', {'variables': ['sla', 'adt', 'ugosa', 'vgosa'], 'tolerances': [1, 0.25, 0.25, 5], 'hasDepth': False, 'isClimatology': False}), ('tblPisces_NRT', {'variables': ['NO3', '

In [29]:
env

{'variables': ['sst'],
 'tolerances': [1, 0.25, 0.25, 5],
 'hasDepth': False,
 'isClimatology': False,
 'startTime': '1981-09-01T00:00:00.000Z',
 'endTime': '2025-06-10T00:00:00.000Z'}

In [30]:
env["startTime"]


'1981-09-01T00:00:00.000Z'

In [35]:
#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

ImportError: cannot import name 'Tracer' from 'IPython.core.debugger' (C:\Users\klongnecker\AppData\Local\miniconda3\Lib\site-packages\IPython\core\debugger.py)

In [36]:
pip install debugger

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for debugger: filename=debugger-1.3-py3-none-any.whl size=3347 sha256=0f6708047f34dbd0085b07dfcebb2b51a54eea836702bf50ee5055e096ef9e61
  Stored in directory: c:\users\klongnecker\appdata\local\pip\cache\wheels\bf\2c\13\dfcf58c96fc0f83b1c06d6c356485417a3ba1a7acfdd73da13
Successfully built debugger
Note: you may need to restart the kernel to use updated packages.


In [15]:
#this will make a file with everything: data/compiled/compiled.csv 
%run compiler.py



            ##########################################################
            #                                                        #
            #                                                        #
            #         Compiling Colocalized Cyano Datasets           #
            #                                                        #
            #                                                        #
            ##########################################################

            
        
